In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from sklearn.utils import shuffle

In [15]:
# populate list Data with the CSV
data = pd.read_csv("../data/A_Z Handwritten Data.csv").astype('float32')

# Split data the X - Our data , and y - the prdict label
x = data.drop('0', axis = 1)
y = data['0']

#Split the data into training and test subsets
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2)

#Turns the 784 columns of grayscale pixel data into 28x28 array of data
train_x = np.reshape(train_x.values, (train_x.shape[0], 28, 28))
test_x = np.reshape(test_x.values, (test_x.shape[0], 28, 28))